In [1]:
!pip install nltk rouge-score pandas

  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
  Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  DEPRECATION: rouge-score is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for rouge-score ... done

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install rouge

  Using cached rouge-1.0.1-py3-none-any.whl (13 kB)

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [25]:
!pip install code-bert-score


  Using cached code_bert_score-0.4.1.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.10.1-cp311-cp311-macosx_11_0_arm64.whl (8.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.6/254.6 kB 6.1 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached fonttools-4.57.0-cp311-cp311-macosx_10_9_universal2.whl (2.8 MB)
  Using cached kiwisolver-1.4.8-cp311-cp311-macosx_11_0_arm64.whl (65 kB)
  DEPRECATION: code-bert-score is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for code-bert-score ... done

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [31]:
# Import necessary libraries

import json
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
import nltk
import code_bert_score

# Download necessary NLTK data
nltk.download("punkt")

# Define a class to evaluate QA performance using BLEU, ROUGE, and CodeBERT
class QAEvaluator:
    def __init__(self):
        # Initialize ROUGE scorer
        self.rouge = Rouge()
        # Smoothing function for BLEU score
        self.smooth = SmoothingFunction().method1

    # Main evaluation method
    def evaluate(self, qa_pairs):
        results = [] # Stores evaluation results for each QA pair
        predictions = [] # List of predicted answers
        references = [] # List of reference answers

        # Extract prediction, reference answer, and the question
        for qa in qa_pairs:
            prediction = qa.get("answer", "")
            reference = qa.get("reference", "")
            question = qa.get("question", "")

            predictions.append(prediction)
            references.append(reference)

            # Compute BLEU score with smoothing
            bleu = sentence_bleu(
                [reference.split()],
                prediction.split(),
                smoothing_function=self.smooth
            )
            # Compute ROUGE scores
            rouge_scores = self.rouge.get_scores(prediction, reference)[0]

            # Append result dictionary for this QA pair
            results.append({
                "question": question,
                "prediction": prediction,
                "reference": reference,
                "bleu": bleu,
                "rouge-1": rouge_scores["rouge-1"]["f"],
                "rouge-2": rouge_scores["rouge-2"]["f"],
                "rouge-l": rouge_scores["rouge-l"]["f"],
            })

            # Compute CodeBERT scores 
            pred_results =code_bert_score.score(cands=predictions, refs=references, lang='python')

             # Append CodeBERT score to each result
        for i, r in enumerate(results):
            r["codebert"] = pred_results[0][i]  # Extracting codebert score for each result

        return results

#  Load JSON file
with open("LLM_generated_v1.json", "r", encoding="utf-8") as f:
    qa_pairs = json.load(f)

# Instantiate evaluator and evaluate the QA pairs
evaluator = QAEvaluator()
results = evaluator.evaluate(qa_pairs)

#  Print sample results 
for r in results:
    print("\nQ:", r["question"])
    print("Prediction:", r["prediction"])
    print("Reference:", r["reference"])
    print("BLEU:", round(r["bleu"], 4), "| ROUGE-L:", round(r["rouge-l"], 4), "| CodeBert:", r["codebert"])

#  Optional: Save to CSV
pd.DataFrame(results).to_csv("qa_evaluation_results.csv", index=False)
print("\nEvaluation saved to 'qa_evaluation_results.csv'")

[nltk_data] Downloading package punkt to /Users/ajamthe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Q: What is a **Tapis tenant**, and what is its significance in the Tapis ecosystem?
Prediction: **A Tapis tenant** is a self-contained instance of the Tapis platform [1, 2]. It represents an organizational unit or deployment of Tapis, often associated with a specific institution or user community, like the 'tacc' tenant hosted at TACC [1, 2]. Each tenant has its own set of users, resources, and security policies, providing isolation and management within the broader Tapis framework [2].
Reference: A Tapis tenant is a logically isolated view of Tapis objects (i.e., the systems, files, actors, etc.). Each tenant has a base url with which to access the tenant
BLEU: 0.0232 | ROUGE-L: 0.2338 | CodeBert: tensor(0.7621)

Q: Write a Python code snippet using the `tapipy` library to **authenticate with a Tapis tenant** given a base URL, username, and password, and then print the **access token**.
Prediction: ```python
from tapipy.tapis import Tapis

# Replace with your Tapis tenant base URL, u